# Prédiction de l'appétence à un produit bancaire - Sélection du modèle

Dans un premier temps, nous avons étudié les données fournies par la banque et avons construit des features permettant d'en extraire de l'information de la manière la plus susceptible d'être utile à un algorithme d'apprentissage.

Ici, nous proposons et justifions un choix d'algorithme particulier, que nous allons entraîner et optimiser.

In [1]:
import model_utilities as mod
### LOAD FROM OTHER FILES ###
mod.settings()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_precision_recall_curve

from src.infrastructure.build_dataset import DataBuilderFactory, DataMerger
from src.domain.cleaning import MissingValueTreatment
from src.domain.build_features import feature_engineering_transformer

###################
## AVERTISSEMENT ##
###################

# Les constantes utilisées dans ce notebook sont hard-codées. Dans les autres fichiers du projet, 
# elles sont extraites des fichiers config/base.py et config/column_names.py.
# Cependant, dans un souci de clarté et de facilité de lecture, nous souhaitons éviter au lecteur 
# de ce notebook de devoir systématiquement se reporter à leurs contenus.

ModuleNotFoundError: No module named 'src'

## 1) Création des données

Deux jeux de données nous ont été fournis : l'un dont chaque ligne correspond à un contact (ou une série de contacts) avec un client, et l'autre apportant des informations contextuelles sur la situation économique du pays et de la période concernés.

Tout d'abord, nous préprocessons ces deux jeux de données séparément. En effet, il est plus facile de traiter les données manquantes dans le fichier de données économiques, qui contient des valeurs mensuelles, que dans le fichier de clients, où chaque ligne est associée à une date précise.

Ce traitement comprend :
- la reconnaissance colonne par colonne des formats de données (entiers, dates, objets...),
- la traduction des mots français afin d'uniformiser la langue à travers les données,
- la suppression de colonnes inutiles,
- la suppression de lignes possédant trop de valeurs manquantes (quand à la fois AGE et JOB_TYPE ne sont pas renseignés),
- pour les données économiques, la complétion des données manquantes par interpolation,
- pour les données relatives aux clients, la correction d'erreurs évidentes.

Ensuite, les données sont agrégées suivant une colonne MOIS/ANNEE qui est crée pendant le traitement et supprimée tout de suite après l'agrégation. 

Enfin, on sépare la cible (y) du reste des données (X).

In [ ]:
CLIENTS_COLUMNS_TO_DROP

# Preprocessing client data
client_builder = DataBuilderFactory(client_file, 
                                    config_client_data, 
                                    ALL_CLIENT_DATA_TRANSLATION)
client_data = client_builder.transform('client')

# Preprocessing eco data
eco_builder = DataBuilderFactory(eco_file, 
                                 config_eco_data)
eco_data = eco_builder.transform('eco')

# Merging files to final dataset
X, y = DataMerger(client_data, eco_data, MERGER_FIELD).transform()

# 2) Création des ensembles d'entraînement et de test

Nous divisons maintenant les données aléatoirement en un ensemble d'apprentissage, qui nous servira pour entraîner le modèle, et un ensemble de test, sur lequel nous l'évaluerons. Nous fixons arbitrairement la valeur de la graine du générateur aléatoire afin d'obtenir des résultats reproductibles.

Etant donné que les deux classes à prédire sont particulièrement déséquilibrées -- il y a environ 9 fois plus d'échantillons négatifs que de positifs, il est important d'utiliser l'option stratify, qui permet de s'assurer qu'une répartition proche sera observée à la fois dans l'ensemble d'entraînement et dans celui de test.

In [ ]:
# Splitting train and test

random_state = 21

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    stratify=y, 
                                                    random_state=random_state)

## 3) Création du pipeline

D'autre part, nous définissons sous forme d'un pipeline scikit-learn l'ensemble des transformations qui doivent être appliquées aux données, puis passées à un classifieur. On distingue trois composantes dans ce pipeline.

#### a) Traitement des valeurs manquantes

Les tâches suivantes sont effectuées à travers la classe MissingValueTreatment :
- complétion de la variable JOB_TYPE par AGE lorsque c'est possible ; on attribue la catégorie "Student" aux moins de 25 ans, la catégorie "Retired" aux plus de 60 ans, et la catégorie majoritaire aux autres (un choix plus précis ne changerait presque rien au reste de notre méthode),
- complétion des valeurs manquantes des autres variables à l'aide de JOB_TYPE (la valeur majoritaire parmi les éléments de la même catégorie professionnelle pour les variables catégoriques, et la médiane pour les variables continues).

#### b) Construction des descripteurs

Les descripteurs suivants sont calculés par le transformeur feature_engineering_transformer :
- une version clippée entre -4000 et 4000 de ACCOUNT_BALANCE, puis réduite par division par 8000,
- des versions clippées entre 0 et 15 de NB_CONTACTS_CURRENT_CAMPAIGN et NB_CONTACTS_BEFORE_CAMPAIGN, puis réduites par division par 15,
- des encodages one-hot des variables HAS_HOUSING_LOAN, HAS_PERSO_LOAN, HAS_DEFAULT, MARITAL_STATUS et EDUCATION,
- l'indicatrice valant 1 quand JOB_TYPE vaut "Retired",
- l'indicatrice valant 1 quand RESULT_LAST_CAMPAIGN vaut "Success",
- l'indicatrice valant 1 quand MARITAL_STATUS vaut "Single",
- les indicatrices valant 1 respectivement quand AGE est supérieur à 25 et 60,
- une version normalisée de la variable AGE par StandardScaler,
- une version encodée par target encoding de la variable MONTH, elle-même extraite de DATE,
- une variable valant 1 si HAS_HOUSING_LOAN et HAS_PERSO_LOAN valent 1, et 0 sinon,
- les variables socio-économiques du tableau supplémentaire, avec une normalisation utilisant StandardScaler.

#### c) Classification

Nous avons choisi d'utiliser un classifieur par forêt aléatoire pour plusieurs raisons :
- notre expérimentation préliminaire nous a montré que les résultats étaient nettement supérieurs avec cette méthode : une dizaine de centième pour l'aire sous la courbe précision-rappel (nous justifierons le choix de ce critère plus loin),
- la bonne interprétabilité de ce modèle grâce à la Mean Decrease Impurity implémentée dans la méthode feature_importances_ de RandomForestClassifier,
- la disponibilité d'outils d'intelligibilité dédiés aux forêts aléatoires, permettant de mieux expliquer notre méthode au client.

In [ ]:
# Constructing global pipeline

steps = [
    ('imputation', MissingValueTreatment()),
    ('feature_engineering', feature_engineering_transformer()),
    ('rf_clf', RandomForestClassifier())
]

pipeline = Pipeline(steps=steps)

## 4) Optimisation du modèle

Une fois ce cadre choisi, on peut optimiser les paramètres de notre forêt aléatoire. Pour ce faire, nous avons d'abord essayé chaque paramètre de RandomForestClassifier et déterminé lesquels avaient un vrai impact, puis avec une approche gloutonne, nous avons recherché des zones plus favorables pour chacun de ces paramètres.

Nous en avons déduit un ensemble de paramètres que nous souhaitons tester avec une recherche aléatoire par RandomizedSearchCV. Nous devons alors définir un critère de scoring adapté à notre problème. Or, dans cette étude, il s'agit de maximiser la proportion d'appels fructueux et le nombre de clients intéressés qui sont effectivement contactés, c'est-à-dire la précision et le rappel. Nous choisissons donc d'optimiser une fonction de ces deux quantités : l'aire sous la courbe précision-rappel, qui correspond à l'argument scoring='average_precision' dans la fonction RandomizedSearchCV. 

Enfin, nous avons utilisé le paramètre class_weight pour donner un poids plus important à la classe minoritaire, inversement proportionnellement à sa représentation dans les données.

In [ ]:
param_distributions = {
    'rf_clf__n_estimators': [220, 240, 260, 280, 300],
    'rf_clf__max_depth': [6, 7, 8, 9, 10],
    'rf_clf__min_samples_leaf': [9, 10, 11, 12]
}
class_weight = {1: 9, 0: 1}
random_state = 21

clf = RandomizedSearchCV(estimator=pipeline, 
                         param_distributions=param_distributions,
                         scoring='average_precision', 
                         random_state=random_state, 
                         cv=5)
clf.fit(X_train, y_train)

## 5) Evaluation du modèle

In [ ]:
penser à utiliser le pickle pour aller plus vite

Notre modèle final est donc une forêt aléatoire implémentée dans RandomForestClassifier, avec pour arguments (hors arguments par défaut) :
- n_estimators = 220,
- max_depth = 10,
- min_samples_leaf = 10.

On trace sa courbe précision-rappel à partir des prédictions de l'algorithme sur les données de test. 

In [ ]:
y_pred = clf.predict(X_test)
plot_precision_recall_curve(clf.best_estimator_, X_test, y_test)
plt.show()
## ajouter titre etc

L'aire sous cette courbe est de 0,46. ## à vérifier

In [ ]:
courbe precision-recall
aire sous la courbe
matrice de confusion
cumulative_gain / lift